In [1]:
import numpy as np
import pandas as pd 
import os 
from collections import defaultdict
import random

I need the functions from the previous task in order to parse pseudoknots. 

The logic, very shortly is:

Send the arc diagram of the sequence to its shape, assign brackets, and then pullback the brackets.
$${\rm s}:{\rm Arcs}\to {\rm Shapes}, \quad {\rm bracketing}:{\rm Shapes}\to {\rm Brackets} $$
$$\Rightarrow {\rm bracketing}^*:{\rm Arcs}\to{\rm Brackets}$$
such that ${\rm bracketing}^*(pair) = {\rm bracketing}(s(pair))$.

A bit more verbose:
1. Consider the rainbow graph or arc diagram (https://en.wikipedia.org/wiki/Arc_diagram) of the primary structure, with the arcs drawn in the upper half plane.
The nodes are elements of the sequence, and they have an arc connecting them if they're paired.
2. Create the "shape", which means that first all the isolated, unconnected nodes are removed. Then, parallel pairs, which is a sequence like ${(A + i, B - i)}_{i\in[0..n]}$ are collapsed or glued to a single ark.
3. Finally, use the shape to assign brackets. Isolated arcs in the "shape" all get "()" pairs. Overlapping arcs get (),<>,[],etc. in a predefined sequence
4. And then make sure everything in the preimage of map 2 has the same bracket (or "no bracket" ie a dot for the unassigned isolated nodes). 


How I implemented the logic was a bit different, in the sense that after step 1., I didn't create the shape immediately, but I first focused on connected components of overlapping/intersecting arcs. 

Then, in each component, I collapsed everything to the shape and assigned brackets.

In [11]:
def prune0arcs(pairs):
    """
    Remove 0-arcs (unconnected pairs) from a list of pairs.
    Rename the surviving pairs so they're sequential
    E.g. (1,x),(2,0),(3,y) -> (f(1),f(x)), --this one died--, (f(3),f(y))
    and f(1)=1, f(3)=2, etc.
    Return the new pairs and the map f.
    """
    first_pairs = [a for a,b in pairs if b!=0]
    rename_pairs_map = { a:(i+1) for i,a in enumerate(sorted(first_pairs))}
    renamed_pairs = [(rename_pairs_map[a],rename_pairs_map[b]) for a,b in pairs if b!=0]
    
    return renamed_pairs, rename_pairs_map

def build_overlap_graph(pairs):
    """
    In a rainbow graph of a primary structure
    where pairs are connected by arcs,
    return a graph in the form of an adjancency list,
    which consists of those pairs whose arcs intersect (this is the pseudoknot condition).
    """
    overlaps = {}
    for i, j in pairs:
        if i < j:  
            overlaps[(i, j)] = []
    for (i1, j1) in pairs:
        for (i2, j2) in pairs:
        # pseudoknot/overlapping arcs condition
            if (i1 < i2 < j1 < j2) or (i2 < i1 < j2 < j1):
                overlaps[(i1, j1)].append((i2, j2))
                overlaps[(i2, j2)].append((i1, j1))
    
    return overlaps

def connected_components(graph):
    """Standard dfs algorithm to find connected components in a graph"""
    visited = set()
    components = []

    def dfs(node, component):
        visited.add(node)
        component.append(node)
        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                dfs(neighbor, component)
    
    for node in graph:
        if node not in visited:
            component = []
            dfs(node, component)
            components.append(component)
    
    return components

def make_dot_bracket(sequence, pairs):
    """
    Creates a dot-bracketed string from a sequence and a list of paris.
    First, remove 0-arcs (unconnected pairs) and rename the pruned sequence.
    Then, build a graph, so that nodes are pairs, and edges connect 
    only those pairs whose arcs intersect (so their topology is nontrivial). 
    Then, focus on connected components of this graph. 
    Give all parallel arcs the same bracket.
    """
    renamed_pairs, rename_pairs_map = prune1arcs(pairs)

    inverse_rename_pairs_map = { v:k for k,v in rename_pairs_map.items()}

    graph = build_overlap_graph(renamed_pairs)
    components = connected_components(graph)
    openbrackets = '(<[{ACEGIKMOQSU'
    closedbrackets = ')>]}BDFHJLNPRTV'
    bracketdict = defaultdict(lambda:'.')

    for component in components:
    
        # e.g. for the component [(1, 6), (2, 7), (4, 9), (5, 8)]
        
        # first sort according to first element
        sorted_renamed_component = sorted( [ (a,b) for a,b in component if a<b], key=lambda x:x[0])
        
        # next, collapse all parallel pairs 
        # which are those like (i,j) and (i+1,j-1) for any i and j
        # to the first pair in such a sequence.
        
        collapsing_parallels_map = {}
        current = sorted_renamed_component[0] # start from first pair
        for k,pair in enumerate(sorted_renamed_component):
            # iterate through pairs and send all parallel pairs to the same target pair 
            if k>1 and sorted_renamed_component[k][1] != sorted_renamed_component[k-1][1] - 1:
                # if not parallel to previous pair, this is a target 
                current = pair
            collapsing_parallels_map[pair] = current
            
        # for our example, 
        # collapsing_parallels_map is {(1, 6): (1, 6), (2, 7): (1, 6), (4, 9): (4, 9), (5, 8): (4, 9)}
        
        # look at the preimage of the collapsing_parallels map
        # e.g. {(1, 6): (1, 6), (2, 7): (1, 6), (4, 9): (4, 9), (5, 8): (4, 9)}
        d = [ (n, [k for k in collapsing_parallels_map.keys() if collapsing_parallels_map[k] == n]) for n in set(collapsing_parallels_map.values()) ]

        # assign the same bracket to everything in a given preimage
        for k,(target,preimage) in enumerate(sorted(d,key=lambda x:x[1][0])):
            for image in preimage:
                bracketdict[ inverse_rename_pairs_map[image[0]]]= openbrackets[k]
                bracketdict[ inverse_rename_pairs_map[image[1]]]= closedbrackets[k]
           
    return ''.join( bracketdict[i] for i in range(1,len(sequence)+1))

def ct2dbn(source_filename,target_filename):
    """
    Converts .ct file to .dbn file. 
    """
    ct_dir = os.path.join(os.getcwd(),'archiveII')
    file_loc = os.path.join(ct_dir,source_filename)

    print('reading:')
    with open(file_loc,'r') as file:
        lines = file.readlines()
        name = lines[0].split()[1]
        print( '> ' + name )
        sequence = ''.join( line.split()[1] for line in lines[1:] )

        print( ' '.join( line.split()[0] for line in lines[1:] ) )
        print( ' '.join( line.split()[4] for line in lines[1:] ) )
        pairings = [ ( line.split()[0],line.split()[4]) for line in lines[1:]]
        
    pairs = [ (int(a),int(b)) for a,b in pairings ]
    
    print('writing...')
    with open( os.path.join(ct_dir,target_filename),'w') as file:
        file.write('>'+name)
        file.write(sequence+'\n')
        file.write(make_dot_bracket(sequence, pairs))
    print('wrote:')
    with open( os.path.join(ct_dir,target_filename),'r') as file:
        print(file.read())
        


ct 2 fasta:

In [6]:
def ct_to_fasta(input_folder, output_fasta):
    """
    Converts .ct files in a folder to a FASTA file without duplicate sequences.
    """
    sequences = {}  # duplicate sequences will be removed

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".ct"):
            file_path = os.path.join(input_folder, file_name)
            with open(file_path, 'r') as f:
                lines = f.readlines()
                
                sequence = ''.join( line.split()[1] for line in lines[1:] )
                
                if sequence not in sequences:
                    # name unique sequences according to file name
                    sequences[sequence] = file_name  

    # write to FASTA
    with open(output_fasta, 'w') as out_f:
        for i, (seq, source) in enumerate(sequences.items(), start=1):
            out_f.write(f">sequence{i} {source}\n")
            out_f.write(f"{seq}\n")

    print(f"FASTA file created: {output_fasta}, with {len(sequences)} unique sequences.")


In [ ]:
ct_to_fasta("archiveII", "archiveII.fasta")

Will also need to encode "has_pseudoknot" as a target variable for all these sequences.

Use the make_dot_bracket to find the dot-bracket, then flag if a "<" exists in the dot-bracket. This is a bit of an overkill, the overlaps are enough, but I'm lazy to rewrite.

In [22]:
def fasta_to_pk(input_folder, input_fasta, output):
    
    with open(input_fasta, 'r') as file:
        lines = file.readlines()

    lines_to_write = ''
    for line in lines:
        if line.startswith('>'):
            file_name = line.split()[1]
            file_path = os.path.join(input_folder, file_name)
            with open(file_path, 'r') as f:
                ctlines = f.readlines()
                sequence = ''.join( ctline.split()[1] for ctline in ctlines[1:] )
                pairings = [ ( ctline.split()[0],ctline.split()[4]) for ctline in ctlines[1:]]
        
            pairs = [ (int(a),int(b)) for a,b in pairings ]
            dot_bracket = make_dot_bracket(sequence,pairs)
            has_pk = 1 if '<' in dot_bracket else 0
            lines_to_write += line.split()[0][1:] +' '+ str(has_pk) + '\n'
                
    with open(output,'w') as f:
        f.write(lines_to_write)


In [23]:
fasta_to_pk('archiveII', 'archiveII.fasta', 'pk.txt')

In [9]:
def split_clusters(tsv_file, output_dir, train_frac=0.7, val_frac=0.15, test_frac=0.15, seed=42):
    """
    Splits RNA sequences into training, validation, and test sets based on clusters
    from MMseqs2.
    
    Parameters:
    - tsv_file (str): Path to the MMseqs2 output .tsv file with clusters.
    - output_dir (str): Directory to save the split files.
    - train_frac (float): Fraction of clusters for training set.
    - val_frac (float): Fraction of clusters for validation set.
    - test_frac (float): Fraction of clusters for test set.
    - seed (int): Random seed for reproducibility.
    """
    random.seed(seed)
    
    # Read the .tsv file and assign sequences to clusters
    clusters = {}
    with open(tsv_file, 'r') as f:
        for line in f:
            cluster_id, sequence_id = line.strip().split("\t")
            if cluster_id not in clusters:
                clusters[cluster_id] = []
            clusters[cluster_id].append(sequence_id)
    
    # Shuffle clusters into train,val,test sets of clusters
    cluster_ids = list(clusters.keys())
    random.shuffle(cluster_ids)
    
    num_clusters = len(cluster_ids)
    train_end = int(train_frac * num_clusters)
    val_end = train_end + int(val_frac * num_clusters)
    
    train_clusters = cluster_ids[:train_end]
    val_clusters = cluster_ids[train_end:val_end]
    test_clusters = cluster_ids[val_end:]
    
    # Assign the actual sequences to those clusters
    train_set = [seq for cluster in train_clusters for seq in clusters[cluster]]
    val_set = [seq for cluster in val_clusters for seq in clusters[cluster]]
    test_set = [seq for cluster in test_clusters for seq in clusters[cluster]]
    
    # Save to files
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, "train.txt"), 'w') as f:
        f.write("\n".join(train_set) + "\n")
    with open(os.path.join(output_dir, "validation.txt"), 'w') as f:
        f.write("\n".join(val_set) + "\n")
    with open(os.path.join(output_dir, "test.txt"), 'w') as f:
        f.write("\n".join(test_set) + "\n")
    
    print(f"Data split, files saved in {output_dir}")

In [ ]:
split_clusters("clusterRes_cluster.tsv", "data_splits")

I want a slightly different input for my models.

A .csv with rows like

[sequenceID],[letters],[has_pseudoknot]

sequence1,ACGUCUGU...,0


In [57]:
def make_csv(target_file, input_fasta, split_dir, split_file, output):
    with open( os.path.join(os.getcwd(),split_dir,split_file), 'r') as f:
        split_set = [line.strip() for line in f.readlines()]
    
    with open(target_file,'r') as f:
        lines = f.readlines()
        target_dict = dict([line.split()  for line in lines ])
        
    sequence_dict = {}
    with open(input_fasta,'r') as f:
        for line in f.readlines():
            if line.startswith('>'):
                seq_name = line.split()[0][1:]
            else: 
                sequence = line.strip()
                sequence_dict[seq_name]=sequence
                
    with open( os.path.join(os.getcwd(),split_dir,output),'w') as f:
        for seq in split_set:
            f.write(seq+','+sequence_dict[seq]+','+target_dict[seq]+'\n')
        

In [60]:
make_csv('pk.txt', 'archiveII.fasta', 'data_splits','test.txt', 'test.csv')
make_csv('pk.txt', 'archiveII.fasta', 'data_splits','train.txt', 'train.csv')
make_csv('pk.txt', 'archiveII.fasta', 'data_splits','validation.txt', 'validation.csv')